In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [2]:
train_normal = pd.read_csv('/Users/gajendersharma/data/TE_dataset/normal_training.csv')
train_attack = pd.read_csv('/Users/gajendersharma/data/TE_dataset/attack_training.csv')


In [3]:
train_normal.columns

Index(['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2',
       'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
       'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
       'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20',
       'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26',
       'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
       'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38',
       'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4',
       'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11',
       'LABEL'],
      dtype='object')

In [4]:
sensors = [col for col in train_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample','LABEL']]


In [5]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_normal

,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,-0.003307,0.300365,0.514600,-1.342734,-0.062398,0.294727,-0.098048,-0.252914,0.522187,0.089759,...,-0.489007,0.005770,1.001364,-0.150228,-0.078897,1.424530,0.520012,-0.165135,0.284192,0.224294
1,0.019690,-0.128967,1.218406,0.925081,-0.856781,1.091018,-0.005032,-0.000158,0.522187,-0.068269,...,-1.191640,-0.016944,-1.639731,-0.250199,0.078954,0.153478,-1.208819,-0.195688,0.470947,-0.631770
2,-0.003307,-0.102502,-0.791012,1.114066,-0.128597,-1.224633,0.154425,-0.422648,1.045516,-0.113762,...,0.816186,0.008733,-0.018895,0.308128,0.123494,0.300514,0.070048,-0.180596,0.175098,1.647425
3,-0.023065,-0.073095,0.083646,1.522366,-0.681827,-1.256668,0.287305,0.413107,-0.524469,-0.121743,...,-0.058911,0.028155,-1.159751,0.113845,0.132008,-0.009071,0.428148,-0.110653,0.996079,-0.020296
4,1.411178,0.447396,-0.301407,-0.105001,-0.062398,1.429671,0.008256,0.715676,-0.524469,-0.856813,...,-0.672117,1.368627,-0.467519,-0.508614,-0.596992,1.300426,0.472379,-0.225506,0.744607,0.233859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,1.385266,-0.696510,-0.913413,-1.775532,-0.166424,0.189470,-0.642856,0.983192,0.522187,0.274923,...,-1.159702,1.517422,-0.422496,-1.291404,0.259076,-1.360723,1.028667,0.122366,-0.540488,0.435407
249996,1.323400,-1.119961,-0.066805,-1.588880,-0.062398,-0.213252,-0.589704,1.724856,-0.524469,1.331635,...,-0.305897,1.427223,0.175670,-0.880204,1.772750,1.533121,1.922641,0.232066,-0.316752,0.848067
249997,1.421867,-1.869822,-0.778261,-1.766199,-0.388662,0.505241,-0.629568,-0.542569,0.522187,1.759428,...,0.403124,1.328795,0.287424,-1.376285,1.345044,0.563560,0.100244,0.253785,0.666947,0.837135
249998,0.071514,0.579725,-0.571710,-0.745449,1.351414,1.919344,-1.014920,0.186180,-0.524469,-0.986906,...,-1.108601,0.082143,0.213457,-1.555478,-0.472544,1.408005,-1.238164,0.308634,-2.347016,0.268703


In [6]:
X_attack=pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])
X_attack 

,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,-0.026325,-0.029518,-0.433862,-0.245717,0.042660,-0.024207,-0.263644,0.319832,-0.268108,-0.125854,...,-0.029945,-0.265605,-0.135945,-0.047142,0.028177,0.337436,0.916798,-0.192790,-0.159342,0.219822
1,-0.041180,0.213434,0.177813,-0.234086,-0.826799,-0.479972,-0.271191,0.567494,0.274182,-0.115952,...,-0.157480,-0.255099,-0.188132,-0.048786,0.020530,-0.216979,-1.134234,-0.200646,-0.059526,0.090487
2,-0.035211,0.076211,0.313438,-0.056163,-1.246538,-0.333849,-0.271191,-0.251499,-0.268108,-0.131342,...,-0.185874,-0.258960,-0.279704,-0.042017,0.032225,-0.051344,-0.317875,-0.197848,-0.083939,0.181265
3,-0.043893,-0.258972,-0.044283,-0.000837,-0.672609,-0.772218,-0.274965,0.388153,0.274182,-0.098415,...,-0.111320,-0.252754,-0.053938,-0.034763,0.007034,0.320676,0.082758,-0.190853,-0.150945,-0.229923
4,-0.382490,-0.333207,1.064378,-0.122806,1.417519,0.229768,-0.316476,-0.523073,0.138609,-0.235253,...,-0.111002,-0.501122,-0.451173,-0.039212,-0.055642,-0.346463,-1.235578,-0.193920,-0.125580,0.248617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.349593,0.064963,0.635660,0.046316,0.453833,0.205414,-0.118984,0.014098,0.003037,0.066935,...,0.003050,-0.467309,-0.338923,0.054983,0.161403,0.318704,-1.145016,0.048963,-0.109477,0.107813
9996,-0.684460,-0.688636,0.550098,-0.302301,0.338190,-0.027686,-0.193201,0.480386,-0.132536,-0.026238,...,-0.079912,-0.688216,-0.147058,0.023263,0.046095,0.258563,-0.616301,0.017217,-0.143500,0.320606
9997,-0.658143,-0.427688,0.797681,-0.141353,-0.407060,-0.584345,-0.195716,0.170381,-0.403681,-0.052842,...,-0.131624,-0.706783,-0.437107,0.047150,0.066413,-1.032005,-1.408942,-0.006727,-0.191979,-0.247006
9998,-0.424136,1.108755,0.492753,-0.192906,0.398153,0.076687,-0.210811,1.229350,0.274182,-0.085889,...,-0.080388,-0.546368,-0.378309,0.015042,0.046395,0.198422,-0.286825,-0.042562,-0.115624,0.597335


In [7]:
hankel = Hankel()
lag = 360
stride = 0.5

In [8]:
df_test1 = pd.read_csv('/Users/gajendersharma/data/TE_dataset/normal_testing.csv')
df_test2 = pd.read_csv('/Users/gajendersharma/data/TE_dataset/attack_testing.csv')

In [9]:
df_test=pd.concat((df_test1,df_test2),axis=0)
df_test

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11,LABEL
0,0,123.0,1,0.25033,3664.1,4535.8,9.5019,26.678,42.184,2705.2,...,24.663,59.713,22.347,39.781,38.365,47.302,47.544,40.490,15.479,0
1,0,123.0,2,0.24936,3637.8,4470.8,9.2864,26.955,42.281,2704.9,...,24.758,60.807,22.107,39.922,37.459,44.229,47.672,41.406,17.832,0
2,0,123.0,3,0.24967,3675.4,4505.9,9.3212,26.892,42.173,2701.8,...,24.735,59.566,22.054,40.325,36.469,46.349,47.514,40.539,17.230,0
3,0,123.0,4,0.24948,3691.1,4496.2,9.4538,26.590,42.311,2701.8,...,24.754,61.175,21.898,40.183,39.098,43.665,47.394,41.493,17.045,0
4,0,123.0,5,0.27535,3684.7,4507.8,9.4292,26.944,42.251,2705.9,...,26.796,61.460,22.516,40.108,37.074,47.643,47.401,40.857,16.861,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,20,123.0,956,0.24154,3670.7,4521.9,9.3236,26.694,42.476,2708.7,...,24.038,61.236,21.076,37.862,28.912,53.094,46.573,41.674,19.470,1
19196,20,123.0,957,0.25582,3621.2,4548.8,9.2826,26.878,41.811,2707.7,...,25.031,58.081,20.684,39.926,38.384,47.338,46.368,40.025,19.312,1
19197,20,123.0,958,0.25370,3615.9,4548.6,9.3915,26.655,42.446,2706.5,...,25.240,60.924,19.844,40.094,38.249,45.703,46.175,40.477,16.401,1
19198,20,123.0,959,0.24663,3702.9,4522.0,9.2319,26.646,42.112,2708.5,...,24.431,59.981,19.370,39.797,41.978,46.439,45.919,41.052,18.051,1


In [10]:
#_________###

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [11]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
Y_test = test_combined.loc[:,'LABEL']

In [12]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [13]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    print(sens,"\n")
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

xmeas_1 

xmeas_2 

xmeas_3 

xmeas_4 

xmeas_5 

xmeas_6 

xmeas_7 

xmeas_8 

xmeas_9 

xmeas_10 

xmeas_11 

xmeas_12 

xmeas_13 

xmeas_14 

xmeas_15 

xmeas_16 

xmeas_17 

xmeas_18 

xmeas_19 

xmeas_20 

xmeas_21 

xmeas_22 

xmeas_23 

xmeas_24 

xmeas_25 

xmeas_26 

xmeas_27 

xmeas_28 

xmeas_29 

xmeas_30 

xmeas_31 

xmeas_32 

xmeas_33 

xmeas_34 

xmeas_35 

xmeas_36 

xmeas_37 

xmeas_38 

xmeas_39 

xmeas_40 

xmeas_41 

xmv_1 

xmv_2 

xmv_3 

xmv_4 

xmv_5 

xmv_6 

xmv_7 

xmv_8 

xmv_9 

xmv_10 

xmv_11 



In [14]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [15]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.963855421686747
Precision  0.975609756097561
Recall  0.9876543209876543
F1-score  0.9815950920245398


In [16]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.2297265987025023 Median : 0.14457831325301204 Min : 0.07228915662650602 Max : 0.9036144578313253
precision - Mean: 0.9964247620284772 Median : 1.0 Min : 0.9663865546218487 Max : 1.0
recall -    Mean: 0.21379392212725548 Median : 0.12345679012345678 Min : 0.04938271604938271 Max : 0.9259259259259259
f1 -        Mean: 0.2991497427971798 Median : 0.21978021978021975 Min : 0.09411764705882353 Max : 0.949367088607595


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [17]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    print(sens)
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=False,kscore_init='inertia')
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

xmeas_1
xmeas_2
xmeas_3
xmeas_4
xmeas_5
xmeas_6
xmeas_7
xmeas_8
xmeas_9
xmeas_10
xmeas_11
xmeas_12
xmeas_13
xmeas_14
xmeas_15
xmeas_16
xmeas_17
xmeas_18
xmeas_19
xmeas_20
xmeas_21
xmeas_22
xmeas_23
xmeas_24
xmeas_25
xmeas_26
xmeas_27
xmeas_28
xmeas_29
xmeas_30
xmeas_31
xmeas_32
xmeas_33
xmeas_34
xmeas_35
xmeas_36
xmeas_37
xmeas_38
xmeas_39
xmeas_40
xmeas_41
xmv_1
xmv_2
xmv_3
xmv_4
xmv_5
xmv_6
xmv_7
xmv_8
xmv_9
xmv_10
xmv_11


In [18]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [19]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.7650602409638554
Precision  0.992
Recall  0.7654320987654321
F1-score  0.8641114982578397


In [20]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.12812789620018536 Median : 0.1144578313253012 Min : 0.024096385542168676 Max : 0.4939759036144578
precision - Mean: 0.8843749999999999 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.10671889838556507 Median : 0.09259259259259259 Min : 0.0 Max : 0.4876543209876543
f1 -        Mean: 0.18354860073205786 Median : 0.1694915254237288 Min : 0.0 Max : 0.6528925619834711


# Multiple clusters + No threshold tuning (No concat of training and test data)

In [21]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'LABEL']

In [22]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [23]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=False,kscore_init='inertia')
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [24]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [25]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.7909090909090909
Precision  0.9770114942528736
Recall  0.8018867924528302
F1-score  0.8808290155440415


In [26]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.1381118881118881 Median : 0.12727272727272726 Min : 0.03636363636363636 Max : 0.5454545454545454
precision - Mean: 0.883974358974359 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.10595065312046445 Median : 0.09433962264150944 Min : 0.0 Max : 0.5471698113207547
f1 -        Mean: 0.18076853107854943 Median : 0.1724137931034483 Min : 0.0 Max : 0.6987951807228916


# Multiple clusters + Threshold tuning (No concat of training and test data)

In [27]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,kscore_init='inertia')
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [28]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [29]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.9636363636363636
Precision  0.9636363636363636
Recall  1.0
F1-score  0.9814814814814815


In [31]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.6627622377622377 Median : 0.7636363636363637 Min : 0.17272727272727273 Max : 0.9636363636363636
precision - Mean: 0.976861089394035 Median : 0.9728048780487805 Min : 0.9375 Max : 1.0
recall -    Mean: 0.6698113207547168 Median : 0.7735849056603774 Min : 0.14150943396226415 Max : 1.0
f1 -        Mean: 0.7514226591271013 Median : 0.8631818055901825 Min : 0.24793388429752064 Max : 0.9814814814814815
